In [1]:
import numpy as np
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from DEFAULTS import PARENT_PATH
import sim.ap_field_ar as ap
import sim.tele_geo_ar as tg
import sim.pan_mod as pm

In [4]:
# default telescope geometry
tele_geo = tg.TelescopeGeometry()

# position of the receiver feed [m]
P_rx = np.array([0, 209, 0])

# arrays of pointing angles of rays
theta_a, theta_b, theta_N = -np.pi / 2 - 0.28, -np.pi / 2 + 0.28, 100
phi_a, phi_b, phi_N = np.pi / 2 - 0.28, np.pi / 2 + 0.28, 100
theta = np.linspace(theta_a, theta_b, theta_N)
phi = np.linspace(phi_a, phi_b, phi_N)

# create panel offsets randomly
save = 0 
error_rms = 35
adj_1_A = np.random.randn(1092) * error_rms
adj_2_A = np.random.randn(1092) * error_rms
panel_model2 = pm.panel_model_from_adjuster_offsets(
    2, adj_2_A, 1, save
)  # Panel Model on M2
panel_model1 = pm.panel_model_from_adjuster_offsets(
    1, adj_1_A, 1, save
)  # Panel Model on M1

# get parameters from telescope geometry
tg_th_1, tg_th2, tg_z_ap = tele_geo.th_1, tele_geo.th2, tele_geo.z_ap
tg_th_fwhp, tg_F_2 = tele_geo.th_fwhp, tele_geo.F_2

# ray tracing
rxmirror = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
apfield = ap.aperature_fields_from_panel_model(panel_model1, panel_model2, \
                                    P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                    theta, phi, rxmirror
                                    )

In [5]:
# here we show the simulated rays
tele_geo.draw_setup()
tele_geo.draw_rays(apfield)
tele_geo.show_figure()
# tele_geo.figure.write_html('figure_rays.html', auto_open=True)